In [1]:
import time
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize

#Página principal 
table = soup.find("div", {"class":"view-content"})
page_url = findNext('a')['href']

boton de mas  = class="js-pager__items pager__items pager-show-more"

href="/es/node?field_programs_value=All&field_sectors_course_target_id=All&field_topics_course_tid=All&field_competencies_course_tid=All&type=All&field_course_language_tid=All&page=5"
class="addAjaxPage"
title="Ir a la página siguiente"

In [2]:
base_url = "https://cursos.iadb.org/es"
data = {}
data['cursos'] = []

In [3]:
def cursos(base_url, salida):
  headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36"}
  page = requests.get(base_url, headers=headers).content
  soup = BeautifulSoup(page, 'html.parser')
  link_mas = soup.find("a", title="Ir a la página siguiente")
  if link_mas != None:
    link_mas = link_mas["href"]
  div = soup.find("div", id="courseslist").find("div", class_="view-content").find_all("a")
  for a in div:
   links = a["href"]
   salida.append(links)
  time.sleep(0.5)
  return salida, link_mas

In [4]:
salida = list()
lista_cursos , link_mas = cursos(base_url, salida)
print("Página 1")
n=2
while link_mas != None:
  print("Página"+str(n))
  n=n+1
  url_lista = base_url+link_mas
  lista_cursos , link_mas = cursos(url_lista, lista_cursos)

Página 1
Página2
Página3
Página4
Página5
Página6
Página7
Página8
Página9
Página10
Página11
Página12


In [5]:
def info_curso(link_pag, data):
  headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36"}
  page = requests.get(link_pag, headers=headers).content
  soup = BeautifulSoup(page, 'html.parser')

  title = soup.title.string
  link_registro = soup.find("a", class_="cu-btn-yellow es")
  if link_registro != None:
    link_registro = link_registro["href"]
  Tipo = soup.find("div",id="type_of_course") 
  if Tipo != None:
    Tipo= Tipo.string
  Fecha = soup.find(class_="cu-course-type-data font-19")
  Fecha_inicio = None
  Fecha_final = None
  if Fecha != None:
    Fecha= Fecha.children
    Fecha = list(Fecha)
    Fecha_inicio = Fecha[0].string
    Fecha_final = Fecha[2].string
  Idioma = soup.find("div",text="IDIOMA")
  if Idioma != None:
    Idioma=Idioma.next_sibling.next_sibling.string
  Dedicacion = soup.find("div",id="effort_course")
  if Dedicacion != None:
    Dedicacion = Dedicacion.string 
  Descripcion = soup.find("div",id="pgr_ABOUT_THIS_COURSE")
  if Descripcion != None:
    Descripcion = Descripcion.text 
  Audiencia = soup.find("div",id="pgr_AUDIENCE_OF_THE_COURSE")
  if Audiencia != None:
    Audiencia= Audiencia.text
  Costos = soup.find("div",id="pgr_COST___REIMBURSEMENTS")
  if Costos != None:
    Costos= Costos.text

  tags_sector = soup.find(class_="col-xs-12 row info_green info_green_SECTOR")
  sector_lista = []
  if tags_sector != None:
    tags_sector=tags_sector.find_all("div")
    tags_sector=list(tags_sector)[1::]
    for tag in tags_sector:
      sector_lista.append(tag.string)

  tags_tema = soup.find(class_="col-xs-12 row info_green info_green_TOPICS")
  tema_lista = []
  if tags_tema != None:
    tags_tema=tags_tema.find_all("div")
    tags_tema=list(tags_tema)[1::]
    for tag in tags_tema:
      tema_lista.append(tag.string)


  tags_competencias = soup.find(class_="col-xs-12 row info_green info_green_COMPETENCIES")
  competencias_lista = []
  if tags_competencias != None:
    tags_competencias=tags_competencias.find_all("div")
    tags_competencias=list(tags_competencias)[1::]
    for tag in tags_competencias:
      competencias_lista.append(tag.string)


  data['cursos'].append({
      'title': title,
      'link_registro': link_registro,
      'Tipo': Tipo,
      'Fecha_inicio': Fecha_inicio,
      'Fecha_final': Fecha_final,
      'Idioma': Idioma,
      'Dedicacion': Dedicacion,
      'Descripcion': Descripcion,
      'Audiencia': Audiencia,
      'Costos': Costos, 
      'sector_lista': sector_lista,
      'tema_lista': tema_lista,
      'competencias_lista': competencias_lista})
  time.sleep(0.5)
  return data

In [6]:
for curso in lista_cursos:
  link_pag = base_url+curso
  data = info_curso(link_pag, data)

In [7]:
df = pd.DataFrame(data['cursos'])
df

,title,link_registro,Tipo,Fecha_inicio,Fecha_final,Idioma,Dedicacion,Descripcion,Audiencia,Costos,sector_lista,tema_lista,competencias_lista
0,Especialización Diseño y gestión de proyectos ...,https://www.coursera.org/specializations/espec...,MOOC,"Ene 10, 2022","Ene 10, 2025",Español,64 horas (16h cada curso de la especialización),¿Sabes que los proyectos de desarrollo son cru...,Esta especialización está dirigida a personas ...,El acceso al contenido de los cursos es gratis...,"[Agricultura y Desarrollo Social, Educación, E...","[Desarrollo Social, Desarrollo sostenible]","[Efectividad en el Desarrollo, Estrategias de ..."
1,Proyecto final en diseño y gestión de proyecto...,https://www.coursera.org/learn/proyecto-final-...,MOOC,"Ene 20, 2023","Ene 01, 2025",Español,13 horas,Los proyectos de desarrollo son cruciales pa...,Funcionariado público a nivel nacional y subna...,El acceso al contenido es gratis durante 7 día...,"[Agricultura y Desarrollo Social, Energía, Med...",[Desarrollo sostenible],"[Pensamiento crítico, analítico y creativo, Pl..."
2,Gestión con enfoque en riesgos en proyectos de...,https://www.coursera.org/learn/gestion-con-enf...,MOOC,"Abr 21, 2022","Ene 01, 2025",Español,16 horas,¿Sabías que cada decisión que tomamos en nuest...,Funcionariado público a nivel nacional y subna...,El acceso al contenido es gratis durante 7 día...,"[Agricultura y Desarrollo Social, Energía, Med...","[Gestión del riesgo ambiental y social, Desarr...","[Gestión del riesgo social y ambiental, Estrat..."
3,Herramientas de gestión en proyectos de desarr...,https://www.coursera.org/learn/herramientas-de...,MOOC,"Abr 21, 2022","Ene 01, 2025",Español,16 horas,¿Conoces el valor de utilizar distintas herram...,Funcionariado público a nivel nacional y subna...,El acceso al curso es gratis durante 7 días. S...,"[Agricultura y Desarrollo Social, Energía, Med...",[Desarrollo sostenible],"[Efectividad en el Desarrollo, Estrategias de ..."
4,Diseño de proyectos de desarrollo | BID | MOOC,https://www.coursera.org/learn/diseno-de-proye...,MOOC,"Abr 21, 2022","Ene 01, 2025",Español,16 horas,¿Sabes cómo hacer para que tus proyectos logre...,Funcionariado público a nivel nacional y subna...,El acceso al contenido es gratis durante 7 día...,"[Agricultura y Desarrollo Social, Energía, Med...",[Desarrollo sostenible],"[Estrategias de gestión, Planeación & Organiza..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,La Realidad Macroeconómica: Una Introducción a...,https://indesvirtual.iadb.org/course/view.php?...,Recursos Educativos Abiertos,"Ene 01, 2022","Dic 31, 2022",Español,10 horas,"En este curso, se estudian los problemas y pol...",None,Sin costo.,"[Reforma / Modernización del Estado, Comercio]",[Economía y Desarrollo],[Efectividad en el Desarrollo]
137,Movingonline | BID | Curso tutorizado,https://indesvirtual.iadb.org/course/view.php?...,Recursos Educativos Abiertos,"Ene 01, 2022","Dic 31, 2022",Español,3 horas,Desde el grupo BID hemos creado la iniciativa ...,None,Ningún costo.,[Educación],"[Desarrollo de instituciones, Desarrollo Social]","[Comunicación efectiva, Análisis de Problemas ..."
138,"La Realidad Fiscal, una Introducción a los Pro...",https://indesvirtual.iadb.org/course/view.php?...,Recursos Educativos Abiertos,"Ene 01, 2022","Dic 31, 2022",Español,10 horas,Este curso apunta a identificar los problemas ...,None,Sin costo.,"[Reforma / Modernización del Estado, Comercio]",[Economía y Desarrollo],[Efectividad en el Desarrollo]
139,OPEN MUSEUM | BID | Curso tutorizado,https://indesvirtual.iadb.org/course/view.php?...,Recursos Educativos Abiertos,"Ene 01, 2022","Dic 31, 2022",Español,10 horas,En tiempos de distanciamiento social y cierre ...,None,Ninguno. Es totalmente gratuito el acceso a to...,"[Empresa Privada y Desarrollo de las PYMES, Ci...",[Desarrollo de instituciones],"[Efectividad en el Desarrollo, Análisis de Pro..."


In [8]:
df.to_excel('Crawling Bid.xlsx', index=True)